In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-17 23:18:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.65MB/s    in 0.6s    

2021-11-17 23:18:34 (1.65 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DatAnalyticsProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
m_url = "https://dataanalyticsproject.s3.us-east-2.amazonaws.com/Merged_M5000.csv"
spark.sparkContext.addFile(m_url)
Merged_M5000_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
Merged_M5000_df .show(5)

+-----+---------+-----+---------+--------------------+------------+--------------------+----+-----------+-----------+
|  _c0|Area_Code| Area|Item_Code|                Item|Element_Code|             Element|Year| Population|   Emission|
+-----+---------+-----+---------+--------------------+------------+--------------------+----+-----------+-----------+
|21795|     5000|World|     5058|Enteric Fermentation|        7225|     Emissions (CH4)|1990|5327231.061| 89059.6919|
|21796|     5000|World|     5058|Enteric Fermentation|      724413|Emissions (CO2eq)...|1990|5327231.061|2493671.372|
|21797|     5000|World|     5058|Enteric Fermentation|      723113|Emissions (CO2eq)...|1990|5327231.061|2493671.372|
|21798|     5000|World|     5059|   Manure Management|        7225|     Emissions (CH4)|1990|5327231.061|  9261.0769|
|21799|     5000|World|     5059|   Manure Management|        7230|     Emissions (N2O)|1990|5327231.061|   455.0201|
+-----+---------+-----+---------+--------------------+--

In [7]:
# Create the Merged_M5000_Table_df Table DataFrame
Merged_M5000_Table_df = Merged_M5000_df.select(["Area_Code", "Area", "Item_Code", "Item", "Element_Code", "Element", "Year", "Population", "Emission"])
Merged_M5000_Table_df.show(5)

+---------+-----+---------+--------------------+------------+--------------------+----+-----------+-----------+
|Area_Code| Area|Item_Code|                Item|Element_Code|             Element|Year| Population|   Emission|
+---------+-----+---------+--------------------+------------+--------------------+----+-----------+-----------+
|     5000|World|     5058|Enteric Fermentation|        7225|     Emissions (CH4)|1990|5327231.061| 89059.6919|
|     5000|World|     5058|Enteric Fermentation|      724413|Emissions (CO2eq)...|1990|5327231.061|2493671.372|
|     5000|World|     5058|Enteric Fermentation|      723113|Emissions (CO2eq)...|1990|5327231.061|2493671.372|
|     5000|World|     5059|   Manure Management|        7225|     Emissions (CH4)|1990|5327231.061|  9261.0769|
|     5000|World|     5059|   Manure Management|        7230|     Emissions (N2O)|1990|5327231.061|   455.0201|
+---------+-----+---------+--------------------+------------+--------------------+----+-----------+-----

In [8]:
from pyspark import SparkFiles
l_url = "https://dataanalyticsproject.s3.us-east-2.amazonaws.com/Merged_L5000.csv"
spark.sparkContext.addFile(l_url)
Merged_L5000_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
Merged_L5000_df .show(5)

+---+---------+-----------+---------+--------------------+------------+--------------------+----+----------+---------+
|_c0|Area_Code|       Area|Item_Code|                Item|Element_Code|             Element|Year|Population| Emission|
+---+---------+-----------+---------+--------------------+------------+--------------------+----+----------+---------+
|  0|        2|Afghanistan|     5058|Enteric Fermentation|        7225|     Emissions (CH4)|1990| 12412.308| 178.4682|
|  1|        2|Afghanistan|     5058|Enteric Fermentation|      724413|Emissions (CO2eq)...|1990| 12412.308|4997.1108|
|  2|        2|Afghanistan|     5058|Enteric Fermentation|      723113|Emissions (CO2eq)...|1990| 12412.308|4997.1108|
|  3|        2|Afghanistan|     5059|   Manure Management|        7225|     Emissions (CH4)|1990| 12412.308|   8.5165|
|  4|        2|Afghanistan|     5059|   Manure Management|        7230|     Emissions (N2O)|1990| 12412.308|   0.3046|
+---+---------+-----------+---------+-----------

In [9]:
# Create the Merged_L5000_Table_df Table DataFrame
Merged_L5000_Table_df = Merged_L5000_df.select(["Area_Code", "Area", "Item_Code", "Item", "Element_Code", "Element", "Year", "Population", "Emission"])
Merged_L5000_Table_df.show(5)

+---------+-----------+---------+--------------------+------------+--------------------+----+----------+---------+
|Area_Code|       Area|Item_Code|                Item|Element_Code|             Element|Year|Population| Emission|
+---------+-----------+---------+--------------------+------------+--------------------+----+----------+---------+
|        2|Afghanistan|     5058|Enteric Fermentation|        7225|     Emissions (CH4)|1990| 12412.308| 178.4682|
|        2|Afghanistan|     5058|Enteric Fermentation|      724413|Emissions (CO2eq)...|1990| 12412.308|4997.1108|
|        2|Afghanistan|     5058|Enteric Fermentation|      723113|Emissions (CO2eq)...|1990| 12412.308|4997.1108|
|        2|Afghanistan|     5059|   Manure Management|        7225|     Emissions (CH4)|1990| 12412.308|   8.5165|
|        2|Afghanistan|     5059|   Manure Management|        7230|     Emissions (N2O)|1990| 12412.308|   0.3046|
+---------+-----------+---------+--------------------+------------+-------------

**Connect to the AWS RDS instance and write each DataFrame to its table**

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataanalyticsproject.ct0b4s5yoxb0.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "finalproject", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write Merged_M5000_Table_df to table in RDS
Merged_M5000_Table_df.write.jdbc(url=jdbc_url, table='Merged_M5000', mode=mode, properties=config)

In [12]:
# Write Merged_L5000_Table_df to table in RDS
Merged_L5000_Table_df.write.jdbc(url=jdbc_url, table='Merged_L5000', mode=mode, properties=config)